In [ ]:
import os
import sys

import numpy as np
import pandas as pd
import cv2

sys.path.append(r"D:\codes\python\courtship\utils")
from cv2_utils import *

%load_ext autoreload
%autoreload 2

In [ ]:
clips_dir = r"D:\Fight\rotation\Wei Zhang\projects\courtship song\data\out\sleap_out\20220405_0\flyclips_paired"
# clips_dir = r"D:\Fight\rotation\Wei Zhang\projects\courtship song\data\out\sleap_out\20220405_0\flyclips"
# rgb_path = r"D:\Fight\rotation\Wei Zhang\projects\courtship song\data\fromZJC\dvSave-2021_12_29_16_10_58.mp4"
rgb_path = r"D:\Fight\rotation\Wei Zhang\projects\courtship song\data\fromZJC\dvSave-2021_12_29_16_10_58_debg.mp4"

In [ ]:
cap = cv2.VideoCapture(rgb_path)
success = True
frames = []
while(True):
    success, frame = cap.read()
    if not success: break
    frames.append(frame)
frames = np.array(frames)
    

In [ ]:
def get_bbox(xy_head, xy_tail, distance=None, bbox_side_len=None, angle=120/180*np.pi):

    flylen = np.sqrt(np.sum((xy_head - xy_tail)**2))

    if not distance:
        distance = flylen * 1.6

    if not bbox_side_len:
        bbox_side_len = flylen

    xy_head = np.array(xy_head)
    xy_tail = np.array(xy_tail)
    a = (xy_head - xy_tail) / flylen
    n = np.array([a[1], -a[0]])

    b1 = -a * np.cos(np.pi-angle) + n * np.sin(np.pi-angle)
    b2 = -a * np.cos(np.pi-angle) - n * np.sin(np.pi-angle)

    center1 = xy_tail + b1 * distance
    center2 = xy_tail + b2 * distance

    bbox1 = np.array([center1[0] - 0.5*bbox_side_len, center1[1] - 0.5*bbox_side_len, 
                     bbox_side_len, bbox_side_len]).round().astype(int)
    bbox2 = np.array([center2[0] - 0.5*bbox_side_len, center2[1] - 0.5*bbox_side_len, 
                     bbox_side_len, bbox_side_len]).round().astype(int)

    return bbox1, bbox2


In [ ]:
def annotate_frames_add_bbox(frames, clip, paired=True):
    frames_clip = frames[clip['frame'].values,].copy()

    colors = [[255,0,0], [0,255,0], [0,0,255], [255,255,0], [255,0,255], [0,255,255]]

    if paired:
        xy_head_0 = clip[['x_head_0', 'y_head_0']].values.astype(float).round().astype(int)
        xy_head_1 = clip[['x_head_1', 'y_head_1']].values.astype(float).round().astype(int)
        xy_tail_0 = clip[['x_tail_0', 'y_tail_0']].values.astype(float).round().astype(int)
        xy_tail_1 = clip[['x_tail_1', 'y_tail_1']].values.astype(float).round().astype(int)
        flylen_0 = np.sqrt(((xy_head_0 - xy_tail_0)**2).sum(axis=1)).mean().round().astype(int)
        flylen_1 = np.sqrt(((xy_head_1 - xy_tail_1)**2).sum(axis=1)).mean().round().astype(int)

    else:
        xy_head = clip[['x_head', 'y_head']].values.astype(float).round().astype(int)
        xy_tail = clip[['x_tail', 'y_tail']].values.astype(float).round().astype(int)
        flylen = np.sqrt(((xy_head - xy_tail)**2).sum(axis=1)).mean().round().astype(int)

    
    for i in range(len(clip)):
        img = frames_clip[i,]

        # add point
        if paired:
            cv2.circle(img, center=xy_head_0[i,], radius=3, color=colors[0], thickness=-1)
            cv2.circle(img, center=xy_tail_0[i,], radius=3, color=colors[1], thickness=-1)
            cv2.circle(img, center=xy_head_1[i,], radius=3, color=colors[2], thickness=-1)
            cv2.circle(img, center=xy_tail_1[i,], radius=3, color=colors[3], thickness=-1)
        else:
            cv2.circle(img, center=xy_head[i,], radius=3, color=colors[0], thickness=-1)
            cv2.circle(img, center=xy_tail[i,], radius=3, color=colors[1], thickness=-1)

        # add bbox
        if paired:
            bbox1, bbox2 = get_bbox(xy_head_0[i,], xy_tail_0[i,], distance=1.5*flylen_0, bbox_side_len=flylen_0)
            x,y,w,h = bbox1
            cv2.rectangle(img, pt1=(x,y), pt2=(x+w,y+h), color=colors[0], thickness=2)
            x,y,w,h = bbox2
            cv2.rectangle(img, pt1=(x,y), pt2=(x+w,y+h), color=colors[0], thickness=2)

            bbox1, bbox2 = get_bbox(xy_head_1[i,], xy_tail_1[i,], distance=1.5*flylen_1, bbox_side_len=flylen_1)
            x,y,w,h = bbox1
            cv2.rectangle(img, pt1=(x,y), pt2=(x+w,y+h), color=colors[2], thickness=2)
            x,y,w,h = bbox2
            cv2.rectangle(img, pt1=(x,y), pt2=(x+w,y+h), color=colors[2], thickness=2)
        else:
            bbox1, bbox2 = get_bbox(xy_head[i,], xy_tail[i,], distance=1.5*flylen, bbox_side_len=flylen)
            x,y,w,h = bbox1
            cv2.rectangle(img, pt1=(x,y), pt2=(x+w,y+h), color=colors[0], thickness=2)
            x,y,w,h = bbox2
            cv2.rectangle(img, pt1=(x,y), pt2=(x+w,y+h), color=colors[0], thickness=2)

        frames_clip[i,] = img

    return frames_clip



for root, _, files in os.walk(clips_dir):
    for file in files:
        if file.endswith('.txt'):
            clip = pd.read_csv(os.path.join(root, file), index_col=0)
            frames_clip = annotate_frames_add_bbox(frames, clip, paired=True)
            save_frames(os.path.join(root, file.split('.')[0] + '.mp4'), frames_clip)